In [45]:
import numpy as np
import tensorflow as tf
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense


In [46]:
max_features = 10000  # vocabulary size
max_len = 100         # max length for padding
# Load data
tweets = pd.read_csv("Tweets.csv")
tweet = tweets.dropna(subset=['text'])
tweet['negativereason_confidence']=tweet['negativereason_confidence'].fillna(0.5)

In [47]:
# Extract X and y
X = tweet['text']
y = tweet['negativereason_confidence']

# Tokenize the entire dataset (fit tokenizer on all texts)
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X)

In [48]:
# Convert texts to sequences of integers
sequences = tokenizer.texts_to_sequences(X)

In [49]:
# Split sequences and labels into train and test sets
X_train_seq, X_test_seq, y_train, y_test = train_test_split(
    sequences, y, test_size=0.2, random_state=42
)

In [50]:
# Pad the token sequences so they have the same length
X_train = pad_sequences(X_train_seq, maxlen=max_len)
X_test = pad_sequences(X_test_seq, maxlen=max_len)
print(f'Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}')

Training data shape: (11712, 100), Training labels shape: (11712,)
Testing data shape: (2928, 100), Testing labels shape: (2928,)


In [51]:
## Train Simple RNN
model=Sequential()
model.add(Embedding(max_features,128,input_length=max_len)) ## Embedding Layers
model.add(SimpleRNN(128,activation='relu'))
model.add(Dense(1,activation="sigmoid"))

In [52]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [53]:
## Create an instance of EarlyStopping Callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [54]:
## Train the model with early stopping
history=model.fit(
    X_train,y_train,epochs=10,batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping]
)

Epoch 1/10
293/293 [==============================] - 9s 26ms/step - loss: 0.6508 - accuracy: 0.2499 - val_loss: 0.6294 - val_accuracy: 0.2582
Epoch 2/10
293/293 [==============================] - 8s 26ms/step - loss: 0.6116 - accuracy: 0.2935 - val_loss: 0.6326 - val_accuracy: 0.2629
Epoch 3/10
293/293 [==============================] - 8s 28ms/step - loss: 0.5746 - accuracy: 0.3127 - val_loss: 0.6473 - val_accuracy: 0.2539
Epoch 4/10
293/293 [==============================] - 7s 25ms/step - loss: 0.5400 - accuracy: 0.3212 - val_loss: 0.6651 - val_accuracy: 0.2505
Epoch 5/10
293/293 [==============================] - 7s 24ms/step - loss: 0.5132 - accuracy: 0.3267 - val_loss: 0.6871 - val_accuracy: 0.2450
Epoch 6/10
293/293 [==============================] - 7s 24ms/step - loss: 0.4948 - accuracy: 0.3292 - val_loss: 0.7125 - val_accuracy: 0.2501


In [55]:
## Save model file
model.save('TweeterSentimentAnalysis.h5')

c:\Users\Asus-pc\Documents\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
